In [ ]:
#Importing
import numpy as np
import pandas as pd
from scipy import misc
import math
import statsmodels.formula.api as sm
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
!pip install pygam
!pip install progressbar2
from pygam import PoissonGAM, s, te
from pygam import LinearGAM, s, f
from pygam import GAM
r_utils = importr('utils')
r_utils.install_packages('codetools')
r_utils.install_packages('gam')

%matplotlib inline
%load_ext rpy2.ipython

In [ ]:
#Future GAM - Predicting Almond Yield Until 2050

#Loading Data
TRAINING_DATA_URL = "https://raw.githubusercontent.com/carolynsmith29/Almond/main/Almond%20Data%20-%20monte_carlo_data_PAST%20(2).csv"
FUTURE_DATA_URL = "https://raw.githubusercontent.com/carolynsmith29/Almond/main/Almond%20Data%20-%20monte_carlo_data_FUTURE%20(1).csv"
COMBINED_DATA_URL = "https://raw.githubusercontent.com/carolynsmith29/Almond/main/Almond%20Data%20-%20GAM_inputs%20(1).csv"
combined_df = pd.read_csv(COMBINED_DATA_URL,names=["Year", "Almond Acreage", 
                                                   "Non Burnable (acres)", 
                                                   "Burnable but Not Burned (acres)", 
                                                   "Lowest (acres)", 
                                                   "Lower (acres)", "Middle (acres)", 
                                                   "Higher (acres)", "Highest (acres)", 
                                                   "Production (millions of lbs)"], 
                                                    header=None, sep=",", index_col=False, 
                                                    skipinitialspace=True )
future_df = pd.read_csv(FUTURE_DATA_URL, names=["Year", "Almond Acreage", 
                                                "Non Burnable (acres)", 
                                                "Burnable but Not Burned (acres)", 
                                                "Lowest (acres)", 
                                                "Lower (acres)", "Middle (acres)", 
                                                "Higher (acres)", "Highest (acres)"], 
                                                header=None, sep=",", index_col=False, 
                                                skipinitialspace=True)
df = pd.read_csv(TRAINING_DATA_URL, names=["Year", "Almond Acreage", 
                                           "Non Burnable (acres)", 
                                           "Burnable but Not Burned (acres)", 
                                           "Lowest (acres)", "Lower (acres)", 
                                           "Middle (acres)", "Higher (acres)", 
                                           "Highest (acres)", 
                                           "Production (millions of lbs)"], 
                                            header=None, sep=",", index_col=False, 
                                            skipinitialspace=True)

#Populating Vectors R Understands
r_y = robjects.FloatVector(df['Production (millions of lbs)'])
r_year = robjects.FloatVector(df['Year'])
r_aa = robjects.FloatVector(df["Almond Acreage"])
r_nb = robjects.FloatVector(df["Non Burnable (acres)"])
r_bbnb = robjects.FloatVector(df["Burnable but Not Burned (acres)"])
r_lowest = robjects.FloatVector(df["Lowest (acres)"])
r_lower = robjects.FloatVector(df['Lower (acres)'])
r_mid = robjects.FloatVector(df['Middle (acres)'])
r_high = robjects.FloatVector(df['Higher (acres)'])
r_highest = robjects.FloatVector(df['Highest (acres)'])

r_year2 = robjects.FloatVector(future_df['Year'])
r_aa2 = robjects.FloatVector(future_df["Almond Acreage"])
r_nb2 = robjects.FloatVector(future_df["Non Burnable (acres)"])
r_bbnb2 = robjects.FloatVector(future_df["Burnable but Not Burned (acres)"])
r_lowest2 = robjects.FloatVector(future_df["Lowest (acres)"])
r_lower2 = robjects.FloatVector(future_df['Lower (acres)'])
r_mid2 = robjects.FloatVector(future_df['Middle (acres)'])
r_high2 = robjects.FloatVector(future_df['Higher (acres)'])
r_highest2 = robjects.FloatVector(future_df['Highest (acres)'])

#Miscellaneous Set-Up
r_gam_lib = importr('gam')
r_gam = r_gam_lib.gam
r_lm = robjects.r["lm"]
r_predict = robjects.r["predict"]
r_summary = robjects.r["summary"]
r_anova = robjects.r["anova"]
r_deviance = robjects.r["deviance"]

#Populating DataFrames R Understands
df_r = robjects.DataFrame({"y":r_y, "year":r_year, "aa": r_aa, "nb":r_nb, "bbnb": r_bbnb, "lowest":r_lowest, "lower":r_lower, 
                           "middle":r_mid, "higher":r_high, "highest":r_highest})
future_df_r = robjects.DataFrame({"year":r_year2, "aa":r_aa2, "nb":r_nb2, "bbnb": r_bbnb2, "lowest":r_lowest2, "lower":r_lower2, 
                                  "middle":r_mid2, "higher":r_high2, "highest":r_highest2})

#Populating Formulas
almond_fmla = robjects.Formula("y ~ s(year) + s(aa) + s(bbnb) + s(lowest) + s(lower) + s(middle) + s(higher) + s(highest)")
almond_fmla.environment["y"] = r_y
almond_fmla.environment["year"] = r_year
almond_fmla.environment["aa"] = r_aa
almond_fmla.environment["nb"] = r_nb
almond_fmla.environment["bbnb"] = r_bbnb
almond_fmla.environment["lowest"] = r_lowest
almond_fmla.environment["lower"] = r_lower
almond_fmla.environment["middle"] = r_mid
almond_fmla.environment["higher"] = r_high
almond_fmla.environment["highest"] = r_highest

almond_test4_fmla = robjects.Formula("y ~ s(aa) + s(bbnb) + s(lowest)")
almond_test4_fmla.environment["y"] = r_y
almond_test4_fmla.environment["year"] = r_year
almond_test4_fmla.environment["bbnb"] = r_bbnb
almond_test4_fmla.environment["aa"] = r_aa
almond_test4_fmla.environment["lowest"] = r_lowest
almond_test4_fmla.environment["lower"] = r_lower
almond_test4_fmla.environment["middle"] = r_mid

#Running Models in R
almond_test4_gam = r_gam(almond_test4_fmla, family='gaussian')

almond1_gam = r_gam(almond_fmla, future_df_r, family="gaussian")

#Graphing Partial Dependencies (individual smoothed functions)
%R -i almond1_gam plot(almond1_gam, residuals=TRUE, se=TRUE, scale=20);

#Predicting Production in the Future
results_test4 = r_predict(almond_test4_gam, future_df_r, type="response")
print(results_test4)

##Combining Data
for i in range(6, 12):
  combined_df.iloc[i, -1] = results_test4[i-6]
display(combined_df)

X=combined_df.iloc[:, 0]
y=combined_df.iloc[:, -1]

##Data Visualization
plt.scatter(X, y, facecolor='blue', edgecolors="none")
plt.xlabel("Year")
plt.ylabel("Production (millions of lbs)")
plt.title("Year vs. Production (millions of lbs)")
plt.show()

display(combined_df.loc[:, ["Year", "Production (millions of lbs)"]])

##Data Visualization 2.0
xyears = combined_df.iloc[:, 0]
display(xyears)
t = np.linspace(xyears[0], xyears[11], 10)
y1 = combined_df.loc[range(6), ["Production (millions of lbs)"]]
display(y1)
y2 = combined_df.loc[range(6,12), ["Production (millions of lbs)"]]
display(y2)

plt.scatter(np.array([2000,2005,2010,2015,2018,2019]), y1, color='red')
plt.scatter(np.array([2025,2030,2035,2040,2045,2050]), y2, color='blue')
plt.xlabel('Year',fontsize=13)
plt.ylabel('Production (millions of lbs)',fontsize=13)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.legend(['Historical Production','Predicted Production'],prop={'size':12})
plt.plot()
plt.show()

##Error Analysis - Deviance
gam_anova = r_anova(almond_test4_gam)
print(gam_anova)
gam_deviance = r_deviance(almond_test4_gam)
print(gam_deviance)

